# DEI Diversity Metrics report

- KPI 1: Percentage of teams with less than 30% representation of a gender within the team. Gender is limited to 'Female' and 'Male'.
- KPI 2: Percentage of teams with more than 30% representation of on a certain nationality within the team.

- Ruled out: every employee that do not report to a CEO-3 UID and every employee with +2 Job Levels distance with their manager.
  

# Metodology details

- Please note that given the current Workday structure leaders are not included in the KPIs of the teams that they lead (e.g. Pete Jaworski do not count in the KPIs of PD&A team, but he does for Shalini Natara's team)
  
- Teams with more less than 5 people are included in the analysis. Meaning that we can find, for example, 100% Male teams in Teams of only 2 males - and they are part of the KPI. Removing them would imply cutting by more than half the number of teams for the analysis (252 out of 419 teams have less than 5 members)

- Some Job levels are not accurate (e.g. managers with Job Level 0). In cases that the difference of the Job Level of the UID with the manager is negative because of this, it is assumed that the job distance is less than 2 and therefore keep in the KPI analysis. Example: An employee has a JL 4, her Manager has a JL 0, I assumed that the distance is not more than 2 job levels and keep them for analysis. 

- 415 Teams were considered for the Gender analysis, as there are 4 teams with all their members missing gender or being unknown.

- 407 Teams were considered for the Nationality KPI, as for 12 teams their most representative in percentage terms Nationality was 'Unknown'.

# Data details

- Raw Workday/Point dataset: 64,960 employees (filter details below).
- Processed dataset after filtering out everyone that do not report to CEO-3+, and removing employees with +2 Job Levels distance with their manager: 2159 employees
- Teams in Point is determined as "Organizational Unit Title".
- 419 Teams are left with the above restrictions in those 2159 employees.



# Data Source details

Point Power BI filter details:

- Data source: Azure Analysis Services (Point)
- Data extraction date: 05/10/2022 - 17:01:15
- Employee Unique ID: is not black
- Employment Status: Active
- Organisation Level 01 Title: APMM CEO
- Organisation Level 02 Title: A.P.Moller Maersk
- Year: 2022
- Report Month: 2022, September

In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

In [2]:
raw_data = pd.read_excel('raw_data/point_data.xlsx')

In [3]:
print('Number of employees in the Workday dataset:', raw_data.shape[0])

Number of employees in the Workday dataset: 64960


In [146]:
def prepare_point_data(raw_data):

    df = raw_data.copy()

    # Remove the last 3 rows (summary & total rows from the data extract)
    df = df.iloc[:-3]


    # Get Manager Job Levels (Metric not implemented in our HR system "Point")
    manager_job_levels = df[['Employee Unique ID', 'Job Level']]
    manager_job_levels.rename({'Employee Unique ID': 'ManagerUniqueID','Job Level': 'Manager Job Level'}, 
                            axis=1,
                            inplace=True)
    df = df.merge(manager_job_levels, on = 'ManagerUniqueID', how='left')

    # Get CEO level (Metric not implemented in our HR system "Point")
    cond_1 = (df['Organisation Level 03 Title'].isna())
    cond_2 = (~df['Organisation Level 03 Title'].isna() & df['Organisation Level 04 Title'].isna())
    cond_3 = (~df['Organisation Level 03 Title'].isna() & ~df['Organisation Level 04 Title'].isna() & df['Organisation Level 05 Title'].isna())
    cond_4 = (~df['Organisation Level 03 Title'].isna() & ~df['Organisation Level 04 Title'].isna() & ~df['Organisation Level 05 Title'].isna() & df['Organisation Level 06 Title'].isna())

    df['CEO Level'] = np.where(cond_1, 'CEO-1', np.nan)
    df['CEO Level'] = np.where(cond_2, 'CEO-2', df['CEO Level'])
    df['CEO Level'] = np.where(cond_3, 'CEO-3', df['CEO Level'])
    df['CEO Level'] = np.where(cond_4, 'CEO-4', df['CEO Level'])

    # Analysis is capped to teams that report to teams that report to CEO-3 (CEO-4 and up)
    df['CEO Level'] = df['CEO Level'].replace('nan', np.nan)
    df = df[~df['CEO Level'].isna()].reset_index(drop=True)

    # Analysis is capped to -1 or 2 JLs from the head of the team 
    df['Job Level Difference with Manager'] = df['Manager Job Level'] - df['Job Level']
    
    cond_5 = df['Job Level Difference with Manager'] <= 2
    cond_6 = (df['Job Level Difference with Manager'].isna()) # If we don't have the diff, we cannot rule them out
    
    df = df[cond_5 | cond_6]

    # Add Headcount
    df['Headcount'] = 1
    
    # Replace NaN nationalities with an 'Unknown' indicator
    df['Nationality ID'] = df['Nationality ID'].fillna('Unknown')

    # Slice the relevant fields
    df = df[['Headcount', 'Employee Unique ID', 'Gender', 'Nationality ID', 
             'ManagerUniqueID', 'CEO Level', 'Organisational Unit Title', 
             'Job Level', 'Manager Job Level', 'Job Level Difference with Manager']].reset_index(drop=True)

    return df

processed_data = prepare_point_data(raw_data)


In [147]:
print('Number of employees after filtering:', processed_data.shape[0])

Number of employees after filtering: 2159


In [151]:
def gender_kpi(processed_data):
        
    df_filtered = processed_data.copy()
    
    # Limiting the analysis to Female and Male composition, as gathering other genders dataset is 
    # still being implemented in Workday and it is not well-spread. This wwould limit data quality issues as well. 
    df_filtered = df_filtered[df_filtered['Gender'].isin(['Male', 'Female'])]

    table_teams_by_gender = (df_filtered.groupby(['Organisational Unit Title']).Gender
    .value_counts(normalize=True)
    .unstack(fill_value=0)
    .stack()
    .reset_index(name='Percentage of Gender')
    )

    number_teams_less_30 = table_teams_by_gender[table_teams_by_gender['Percentage of Gender'] < .3].shape[0]
    number_teams_total = len(table_teams_by_gender['Organisational Unit Title'].unique())

    kpi_gender = (number_teams_less_30 / number_teams_total) * 100


    # From the less represented gender teams, how many times is 'Female' that gender? 
    teams_less_30 = table_teams_by_gender[table_teams_by_gender['Percentage of Gender'] < .3]
    number_teams_less_30_female = teams_less_30[teams_less_30['Gender'] == 'Female'].shape[0]

    kpi_gender_females_perc = (number_teams_less_30_female/number_teams_less_30) * 100

    # KPI prints
    print(number_teams_less_30, 
            "Teams have less than 30% of one gender within their teams from a total of", 
            number_teams_total, 
            "teams analyzed (",
            round(kpi_gender,2),
            "%)")
        
    print("-> Of the {} teams with less than 30% of one gender representation, ".format(number_teams_less_30),
        "in {} % of the cases the gender underrepresented is 'Female'.".format(round(kpi_gender_females_perc,2)))

In [152]:
gender_kpi(processed_data)

273 Teams have less than 30% of one gender within their teams from a total of 415 teams analyzed ( 65.78 %)
-> Of the 273 teams with less than 30% of one gender representation,  in 77.29 % of the cases the gender underpressented is 'Female'.


In [189]:
def nationality_kpi(processed_data):

    df_filtered = processed_data.copy()

    table_teams_by_nationality = (df_filtered.groupby(['Organisational Unit Title'])['Nationality ID']
                                                    .value_counts(normalize=True)
                                                    .reset_index(name='Percentage of Nationality'))


    # We sort the values by % of the nationalities and keep only the highest, as it is the only one needed for the KPI                                                
    table_teams_by_nationality = table_teams_by_nationality.sort_values(by=['Organisational Unit Title', 'Percentage of Nationality'])
    table_teams_by_nationality = table_teams_by_nationality.drop_duplicates(subset='Organisational Unit Title', keep='last')

    # Removing the teams with 'Unknown' being the highest represented Nationality
    table_teams_by_nationality = table_teams_by_nationality[~(table_teams_by_nationality['Nationality ID'] == 'Unknown')]

    # Count the number of them that are over the 30% KPI thresshold
    teams_more_than_30 = table_teams_by_nationality[table_teams_by_nationality['Percentage of Nationality'] > .3]

    # KPI calculation
    number_teams_more_30 = teams_more_than_30.shape[0]
    number_teams_total = len(table_teams_by_nationality['Organisational Unit Title'].unique())

    kpi_nationality = (number_teams_more_30 / number_teams_total) * 100

    print(number_teams_more_30, 
        "Teams have more than 30% of one nationality within their teams from a total of", 
        number_teams_total, 
        "teams analyzed (",
        round(kpi_nationality,2),
        "%)")



In [190]:
nationality_kpi(processed_data)

335 Teams have more than 30% of one nationality within their teams from a total of 407 teams analyzed ( 82.31 %)
